# Creating Simplicial Complex Class

In [1]:
import numpy as np

In [2]:
class Simplicial_Complex:
    def __init__(self, simplices):
        self.simplices = simplices

In [3]:
class Simplex:
    def __init__(self, vertices):
        self.vertices = vertices
        self.dim = len(vertices)-1
        
    #def __add__(self, rhs):
        # use add_simplex
        # use add_chain
    
    def __eq__(self,rhs):
        if self and rhs:
            return self.vertices == rhs.vertices
        else:
            return False
        
    def __str__(self):
        return f"Simplex({self.vertices})"
    
    def __repr__(self):
        return f"Simplex({self.vertices})"
    
    def boundary(self): # input is n-simplex and output is (n-1)-chain
        n = len(self.vertices)
        simplices = [Simplex(self.vertices[:i] + self.vertices[i+1:]) for i in range(n)]
        coefficients = [(-1)**i for i in range(n)]
        #return simplices  # making sure output is correct, but it wasn't; simplices was list of lists of lists; error
        #return coefficients
        return Chain(simplices,coefficients)

In [4]:
class Chain:
    # should be taking in Simplex types
    def __init__(self, simplices, coefficients): # simplices are of same dimension
        if len(simplices) == 1:
            pass
        else:
            length = len(simplices[0].vertices)
            for lst in simplices[1:]:
                if len(lst.vertices) != length:
                    raise TypeError("Simplices must be of same dimension.")


        distinct_simplices = []
        for simplex in simplices:
            if simplex not in distinct_simplices:
                distinct_simplices.append(simplex)


        coef = [0 for i in range(len(distinct_simplices))]
        for i in range(len(distinct_simplices)):
            simplex = distinct_simplices[i]
            for j in range(len(simplices)):
                if simplices[j] == simplex:
                    coef[i] += coefficients[j]
        self.simplices = distinct_simplices
        self.coefficients = coef
        
    def __str__(self):
        return f"Chain({self.simplices}, {self.coefficients})"
    
    def __repr__(self):
        return f"Simplex({self.vertices})"
    
        
    def add_chain(self, other): # same dim  # also make it so you add a simplex not just a list of vertices
        if len(self.simplices[0].vertices) != len(rhs.simplices[0].vertices): 
            raise TypeError("Chains must be of same dimension.")
        else:
            total_simplices = self.simplices + other.simplices
            total_coefficients = self.coefficients + other.coefficients
            temp_chain = Chain(total_simplices, total_coefficients)
            self.simplices = temp_chain.simplices
            self.coefficients = temp_chain.coefficients
            
    def boundary(self):
        total_simplices = []
        total_coefficients = []
        for i in range(len(self.simplices)):
            simplex = self.simplices[i]
            temp_simplices = simplex.boundary().simplices
            for x in range(len(temp_simplices)):
                total_simplices.append(temp_simplices[x])
            temp_coef = simplex.boundary().coefficients
            new_coef = [self.coefficients[i] * temp_coef[j] for j in range(len(temp_coef))]
            for c in range(len(new_coef)):
                total_coefficients.append(new_coef[c])
        new_chain = Chain(total_simplices, total_coefficients)
        return new_chain
    

In [36]:
class BoundaryMap:
    def __init__(self,sc,n):
        # will look through all the simplices, check n-simplices where n is num of vertices
        d_basis = []
        cd_basis = []
        # store those in list called domain-basis
        for i in range(len(sc.simplices)):
            if sc.simplices[i].dim == n:      #2-simplex has 2 vertices, but 1 dimensional, so dim + 1 gives # vertices
                d_basis.append(sc.simplices[i])
        # and repeat for the n-1 simplices
        for i in range(len(sc.simplices)):
            if sc.simplices[i].dim == n-1:
                cd_basis.append(sc.simplices[i])
        # initialize matrix (or vector) which will contain columns vectors, may be rows at first, so prepare for transposing matrix
        if len(d_basis) == 0:
            mat = np.zeros((len(cd_basis),1))
            self.map = mat
        elif len(cd_basis) == 0:
            mat = np.zeros((1, len(d_basis)))
            self.map = mat
        else:
            mat = np.zeros((len(cd_basis),len(d_basis)))
            # for basis_simplex in domain_basis:
            for basis_simplex in d_basis:
                bd_chain = basis_simplex.boundary()
                vector = np.zeros(len(cd_basis))
                for i in range(len(cd_basis)):
                    if cd_basis[i] in bd_chain.simplices:
                        index = bd_chain.simplices.index(cd_basis[i])
                        vector[i] += bd_chain.coefficients[index]
                # now add coefficient vector to matrix
                mat[:,d_basis.index(basis_simplex)] = vector.T
            self.map = mat

In [38]:
sc = Simplicial_Complex([Simplex([1,2,3]), Simplex([1,2,4]), Simplex([1,2]), Simplex([1,3]), Simplex([1])])
BoundaryMap(sc,0).map

array([[0.]])

In [32]:
sc = Simplicial_Complex([Simplex([1,2,3]), Simplex([1,2,4]), Simplex([1,2]), Simplex([1,3]), Simplex([1])])
np.linalg.matrix_rank(BoundaryMap(sc,2).map)

0
[ 1. -1.]
1
[1. 0.]


2

In [21]:
x = np.zeros((3,3))
x[:,1] = [1,2,3]
x[1,:] = [4,5,6]
print(x)

[[0. 1. 0.]
 [4. 5. 6.]
 [0. 3. 0.]]


In [39]:
sc = Simplicial_Complex([Simplex([1]), Simplex([2]), Simplex([3]), Simplex([4]),
                         Simplex([1,2]), Simplex([1,3]), Simplex([1,4]), Simplex([2,3]), Simplex([2,4]),
                         Simplex([1,2,3]), Simplex([1,2,4])])
BoundaryMap(sc,0).map

array([[0., 0., 0., 0.]])

In [42]:
def betti(sc, n):
    d_basis = []
    for i in range(len(sc.simplices)):
        if sc.simplices[i].dim == n:
            d_basis.append(sc.simplices[i])
    dim_Cn = len(d_basis)
    rank_dn = np.linalg.matrix_rank(BoundaryMap(sc,n).map)
    rank_dn_plus_1 = np.linalg.matrix_rank(BoundaryMap(sc,n+1).map)
    betti = dim_Cn - rank_dn - rank_dn_plus_1
    return betti

In [51]:
sc = Simplicial_Complex([Simplex([1]), Simplex([2]), Simplex([3]), Simplex([4]),
                         Simplex([1,2]), Simplex([1,3]), Simplex([1,4]), Simplex([2,3]), Simplex([2,4]), Simplex([3,4])])
for i in range(4):
    print(betti(sc,i))

1
3


ValueError: zero-size array to reduction operation maximum which has no identity

In [46]:
sc = Simplicial_Complex([Simplex([1]), Simplex([2]), Simplex([3]), Simplex([4]),
                         Simplex([1,2]), Simplex([1,3]), Simplex([1,4]), Simplex([2,3]), Simplex([2,4]), Simplex([3,4]),
                         Simplex([1,2,3]), Simplex([1,2,4])])

In [507]:
Simplex([1,2]) in [Simplex([1,3]), Simplex([1,2])]
index = [Simplex([1,3]), Simplex([1,2])].index(Simplex([1,2]))
index

1

In [500]:
ch = Chain([Simplex([1,2]),Simplex([2,3]),Simplex([1,3]),Simplex([1,3])], [1,2,3,4])
print(ch.simplices[2])
print(ch.coefficients)
print(ch)

Simplex([1, 3])
[1, 2, 7]
Chain([Simplex([1, 2]), Simplex([2, 3]), Simplex([1, 3])], [1, 2, 7])


In [501]:
ch1 = Chain([Simplex([1,2]),Simplex([2,3])], [1,2])
print(ch1.simplices)
print(ch1.coefficients)

[Simplex([1, 2]), Simplex([2, 3])]
[1, 2]


In [502]:
# better than sets method (fixes the problem)

simplices = [Simplex([1,2]),Simplex([1,2]),Simplex([2,3])]
distinct_simp = []
for simplex in simplices:
    if simplex not in distinct_simp:
        distinct_simp.append(simplex)
print(distinct_simp)
#print(distinct_simp[0].vertices)

[Simplex([1, 2]), Simplex([2, 3])]


In [504]:
print(ch.boundary())

Chain([Simplex([2]), Simplex([1]), Simplex([3])], [-1, -8, 9])


In [494]:
s = Simplex([1,2])
print(s)
s.vertices

Simplex([1, 2])


[1, 2]

In [495]:
print(s.boundary().simplices[0])
print(s.boundary().simplices[0].vertices)
print(s.boundary().coefficients)

Simplex([2])
[2]
[1, -1]


In [496]:
# note: order is random but coefficients align correctly with corresponding simplex in lists
g = Simplex([1,2,3,4])
for i in range(len(g.boundary().simplices)):
    print(g.boundary().simplices[i])
print(g.boundary().coefficients)

Simplex([2, 3, 4])
Simplex([1, 3, 4])
Simplex([1, 2, 4])
Simplex([1, 2, 3])
[1, -1, 1, -1]


In [472]:
Simplex([1,2]) == Simplex([1,2])

True